In [1]:
import numpy as np
import bigfloat as bgft
import mpmath as mp
from scipy import integrate as integ
from matplotlib import pyplot as plt
%matplotlib inline

# Question 1.

##(a)

Buoyancy is defined as 

$$ b \equiv f_0 \frac{\psi_1 - \psi_2}{H/2} $$

and the relation between the unperturbed flow and geostrophic streamfunction is

$$ U_1 = - \frac{\partial \psi_1}{\partial y}, \ \ \ \ U_2 = - \frac{\partial \psi_2}{\partial y} $$

so taking the $y$ derivative of equation (1) yields

$$ - \frac{\partial b}{\partial y} = f_0 \frac{- \partial \psi_1 / \partial y + \partial \psi_2 / \partial y}{H/2} = f_0 \frac{U_1 - U_2}{H/2} $$

## (b)

For an unstable condition, we need equation (2) so

$$ - \frac{\partial b}{\partial y} \geq f_0 \frac{\frac{1}{4} \beta L_d^2}{H/2} = \frac{f_0 L_d^2}{2H} \frac{f_0}{a} \cos{\theta} = \frac{f_0^2}{2aH} \frac{N^2 H^2}{f_0^2} \cos{\theta} $$

Now, dividing both sides with $\partial \bar{b}/\partial z = N^2$, we get

$$ \frac{\partial z}{\partial y} \bigg|_\bar{b} \geq \frac{N^2 H}{2a} \frac{1}{N^2} \cos{\theta} $$

Thus, we get

$$ \frac{\partial z}{\partial y} \bigg|_\bar{b}  \geq \frac{H}{2a} \cos{\theta} $$

## (c)

The two-level QGPV equation is

$$ \frac{D}{Dt} \bigg[ \zeta_i + \beta y + \frac{k_d^2}{2} \big( \psi_j - \psi_i \big) \bigg] = 0; \ \ \ \ \ i = 1,2, \ \ \ j = 3 - i $$

where $k_d = \frac{\sqrt{8}}{L_d}$ so the basic PV becomes

$$ \bar{q}_1 = \beta y + \frac{k_d^2}{2} \big( \Psi_2 - \Psi_1 \big) = \beta y + \frac{4}{L_d^2} \big( \Psi_2 - \Psi_1 \big) $$ 
$$ \bar{q}_2 = \beta y - \frac{k_d^2}{2} \big( \Psi_2 - \Psi_1 \big) = \beta y - \frac{4}{L_d^2} \big( \Psi_2 - \Psi_1 \big) $$

From the definition of the streamfunction, $U = - \partial \Psi / \partial y$ so we get

$$ \Psi_1 = - U_1 y, \ \ \ \ \Psi_2 = - U_2 y $$

and thus

$$ \bar{q}_1 = \beta y + \frac{4}{L_d^2} \big( U_1 - U_2 \big) y, \ \ \ \ \bar{q}_2 = \beta y - \frac{4}{L_d^2} \big( U_1 - U_2 \big) y $$ 

Hence, the meridional gradients of the basic QGPV are

$$ \frac{\partial \bar{q}_1}{\partial y} = \beta + \frac{4}{L_d^2} \big( U_1 - U_2 \big), \ \ \ \ \frac{\partial \bar{q}_2}{\partial y} = \beta - \frac{4}{L_d^2} \big( U_1 - U_2 \big) $$

Under unstable conditions, i.e.equation (2), we have

$$ \frac{\partial \bar{q}_1}{\partial y} \geq \beta + \frac{4}{L_d^2} \frac{1}{2} \beta L_d^2 = 3 \beta\ \ \ (>0) $$

$$ \frac{\partial \bar{q}_2}{\partial y} \leq \beta - \frac{4}{L_d^2} \frac{1}{2} \beta L_d^2 = - \beta \ \ \ (<0) $$